In [2]:
import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [38]:
import numpy as np
catDict = {
  "American Restaurant": 0,
  "Bar": 1,
  "Asian Restaurant": 2,
  "European Restaurant": 3,
  "Italian Restaurant": 4,
  "Fast Food Restaurant": 5,
  "Mexican Restaurant": 6,
  "Seafood Restaurant": 7,
  "Coffee Shop": 8,
  "Sandwich Shop": 9
}
reviews = []
for l in readGz("train.json.gz"):
    l["category"] = np.array([0] * 10)
    for k,v in catDict.iteritems():
        if k in l["categories"]:
            l["category"][v] = 1
            
    if sum(l["category"]) == 1:
        l["category"] = np.argmax(l["category"])
        reviews.append(l)
print reviews[0]
print len(reviews)

{'category': 0, 'rating': 4.0, 'reviewHash': 'R567271252', 'businessID': 'B423621081', 'unixReviewTime': 1378865039, 'reviewText': u"I'm a vegetarian, but every so often I want a hotdog with lots of toppings.  And a tall can of beer.  Frank has got that covered.  And they have a cool warehouse space with some pinball machines.  Prices are a little high for hotdogs...fancy hotdogs, but hotdogs nonetheless.  Good location and service, but gets crowded and loud.", 'userID': 'U985379327', 'reviewTime': u'Sep 10, 2013', 'categories': [u'American Restaurant', u'Cafe', u'Hot Dog Restaurant'], 'categoryID': 0}
70195


In [39]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

In [44]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

In [41]:
all_words = []
review_hash = {}
i = 0
for r in reviews:
    i = i + 1
    if i%10000 == 0:
        print i/float(len(reviews))
    curr_words = []
    for word in tokenizer.tokenize(r["reviewText"]):
        s = stemmer.stem(word)
        all_words.append(s)
        curr_words.append(s)
    review_hash[r["reviewHash"]] = (curr_words,r["category"])
    

0.142460289194
0.284920578389
0.427380867583
0.569841156778
0.712301445972
0.854761735166
0.997222024361


In [45]:
import nltk
f = nltk.FreqDist(all_words)
all_feats = [i[0] for i in f.most_common(3000)]
print all_feats

[u'.', u'the', u',', u'and', u'a', u'to', u'is', u'i', u'it', u'of', u'for', u'in', u'was', u'food', u'!', u'but', u'you', u'good', u'great', u'place', u'with', u'this', u'are', u'that', u'on', u'they', u'have', u'not', u'my', u'be', u'veri', u'here', u'get', u'go', u'at', u'servic', u'as', u'so', u'had', u'there', u'their', u'if', u'like', u'we', u')', u'(', u'best', u'-', u'one', u'love', u'were', u'realli', u'time', u'all', u'out', u'friend', u'or', u'can', u'just', u'drink', u'restaur', u'"', u'bar', u'nice', u'tri', u'your', u'from', u'an', u'some', u'delici', u'order', u'too', u'also', u'alway', u'price', u'beer', u'menu', u'when', u'make', u'back', u'come', u'...', u'wait', u'up', u'more', u'which', u'has', u'amaz', u'pretti', u'coffe', u'well', u'littl', u'what', u'would', u'about', u'me', u'atmospher', u"don't", u'will', u"i'v", u'onli', u'been', u'excel', u'by', u'dish', u'favorit', u'eat', u'even', u'other', u'night', u'fri', u'staff', u'than', u'chicken', u'tabl', u'burger'

In [52]:
def make_features(text):
    present_features = []
    for w in all_feats:
        present_features.append(w in text)
    return present_features

X = []
y = []
i = 0
for k,v in review_hash.iteritems():
    i += 1
    if i % 1000 == 0:
        print i/(len(review_hash)*1.0)
    X.append(make_features(v[0]))
    y.append(v[1])

0.0142460289194
0.0284920578389
0.0427380867583
0.0569841156778
0.0712301445972
0.0854761735166
0.0997222024361
0.113968231356
0.128214260275
0.142460289194
0.156706318114
0.170952347033
0.185198375953
0.199444404872
0.213690433792
0.227936462711
0.24218249163
0.25642852055
0.270674549469
0.284920578389
0.299166607308
0.313412636228
0.327658665147
0.341904694067
0.356150722986
0.370396751905
0.384642780825
0.398888809744
0.413134838664
0.427380867583
0.441626896503
0.455872925422
0.470118954341
0.484364983261
0.49861101218
0.5128570411
0.527103070019
0.541349098939
0.555595127858
0.569841156778
0.584087185697
0.598333214616
0.612579243536
0.626825272455
0.641071301375
0.655317330294
0.669563359214
0.683809388133
0.698055417052
0.712301445972
0.726547474891
0.740793503811
0.75503953273
0.76928556165
0.783531590569
0.797777619489
0.812023648408
0.826269677327
0.840515706247
0.854761735166
0.869007764086
0.883253793005
0.897499821925
0.911745850844
0.925991879764
0.940237908683
0.95448393

In [ ]:
print X[0]

{'category': 0, 'rating': 4.0, 'reviewHash': 'R567271252', 'businessID': 'B423621081', 'unixReviewTime': 1378865039, 'reviewText': u"I'm a vegetarian, but every so often I want a hotdog with lots of toppings.  And a tall can of beer.  Frank has got that covered.  And they have a cool warehouse space with some pinball machines.  Prices are a little high for hotdogs...fancy hotdogs, but hotdogs nonetheless.  Good location and service, but gets crowded and loud.", 'userID': 'U985379327', 'reviewTime': u'Sep 10, 2013', 'categories': [u'American Restaurant', u'Cafe', u'Hot Dog Restaurant'], 'categoryID': 0}


In [54]:
from sklearn.neural_network import MLPClassifier
nn_class = MLPClassifier(early_stopping = True, verbose = True)

In [55]:
from sklearn.naive_bayes import MultinomialNB
mnb_class = MultinomialNB()

In [71]:
from sklearn.ensemble import VotingClassifier
v_class = VotingClassifier(estimators = [("nn",MLPClassifier(early_stopping = True, verbose = True)), 
                                         ("mnb",MultinomialNB())], voting='soft')

In [56]:
print len(X)

70195


In [57]:
X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

In [59]:
mnb_class.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [60]:
outputs = mnb_class.predict(X_test)
correct = 0
for o,t in zip(outputs,y_test):
    correct += (o==t)
print correct, "/", len(y_test)

6322 / 10195


In [61]:
nn_class.fit(X_train,y_train)

Iteration 1, loss = 1.47546717
Validation score: 0.626000
Iteration 2, loss = 1.08291935
Validation score: 0.636833
Iteration 3, loss = 0.99275261
Validation score: 0.637000
Iteration 4, loss = 0.93406947
Validation score: 0.633500
Iteration 5, loss = 0.88269941
Validation score: 0.631167
Iteration 6, loss = 0.83707802
Validation score: 0.624333
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [62]:
outputs = nn_class.predict(X_test)
correct = 0
for o,t in zip(outputs,y_test):
    correct += (o==t)
print correct, "/", len(y_test)

6443 / 10195


In [73]:
v_class.fit(X_train,y_train)

Iteration 1, loss = 1.49319121
Validation score: 0.624833
Iteration 2, loss = 1.08785108
Validation score: 0.635000
Iteration 3, loss = 0.99743166
Validation score: 0.630667
Iteration 4, loss = 0.93962779
Validation score: 0.629667
Iteration 5, loss = 0.89012936
Validation score: 0.624667
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


VotingClassifier(estimators=[('nn', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum...rbose=True, warm_start=False)), ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [74]:
outputs = v_class.predict(X_test)
correct = 0
for o,t in zip(outputs,y_test):
    correct += (o==t)
print correct, "/", len(y_test)

6468 / 10195


In [77]:
predictions = open("predictions_Category.txt", 'w')
predictions.write("userID-reviewHash,category\n")
for l in readGz("test_Category.json.gz"):
  
    words = tokenizer.tokenize(l['reviewText'])
    words = [stemmer.stem(w) for w in words]
    feats = [make_features(words)]
    cat = v_class.predict(feats)[0]
    predictions.write(l['userID'] + '-' + l['reviewHash'] + "," + str(cat) + "\n")

predictions.close()

In [4]:
l = readGz("test_Category.json.gz")

{'rating': 5.0, 'reviewText': u"I've been here with a large group, eating with the family, and attending the moisture festival. Each time has been a wonderful experience. The beer at Hales has variety and taste. Love their Ntiro tap and their delicious food. Good bar section and separate seating if you're looking for something quieter.", 'userID': 'U784206181', 'reviewHash': 'R287025050', 'unixReviewTime': 1394254315, 'reviewTime': u'Mar 7, 2014'}


In [7]:
print l.next()

{'rating': 2.0, 'reviewText': u"My friend is the head bartender there, so that's great. I think the drinks are over-priced, but it's a fun environment.", 'userID': 'U977431350', 'reviewHash': 'R718278791', 'unixReviewTime': 1376186613, 'reviewTime': u'Aug 10, 2013'}
